In [8]:
import os
import glob
import note_seq
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
import numpy as np
from note_seq import midi_io
from note_seq.protobuf import music_pb2
from collections import defaultdict

Instructions for updating:
non-resource variables are not supported in the long term


## Step 1: Preprocess MIDI Files into Melodies

In [ ]:
# Configuration for data preprocessing
artists_subdir = 'Artist_MIDI'

ARTIST_FOLDERS = {
    'ABBA': f'./{artists_subdir}/ABBA',
}

TRAINING_DATA_DIR = f'./{artists_subdir}/training_data'
MIN_NOTES = 16  # Minimum notes required in a melody
MAX_NOTES = 1000  # Maximum notes to extract
TRANSPOSE_RANGE = 12  # Transpose to all 12 keys for augmentation

os.makedirs(TRAINING_DATA_DIR, exist_ok=True)

def extract_melody_from_sequence(sequence, min_notes=MIN_NOTES, max_notes=MAX_NOTES):
    """
    Extract the melody (highest pitched notes) from a NoteSequence.
    
    Args:
        sequence: note_seq.NoteSequence object
        min_notes: minimum notes required
        max_notes: maximum notes to extract
        
    Returns:
        note_seq.Melody object or None if invalid
    """
    try:
        # Extract single melody line
        melody = note_seq.Melody()
        sorted_notes = sorted(sequence.notes, key=lambda n: n.start_time)
        
        if len(sorted_notes) < min_notes:
            return None
            
        # Build melody from highest notes at each time step
        for note in sorted_notes[:max_notes]:
            melody.append(note.pitch)
        
        if len(melody) < min_notes:
            return None
            
        return melody
        
    except Exception as e:
        return None

def transpose_melody(melody, semitones):
    """Transpose melody by given semitones (positive or negative)."""
    return note_seq.Melody([max(0, min(127, pitch + semitones)) for pitch in melody])

print("🎼 MIDI Preprocessing Configuration")
print(f"  Artists to process: {list(ARTIST_FOLDERS.keys())}")
print(f"  Output directory: {TRAINING_DATA_DIR}")
print(f"  Min notes per melody: {MIN_NOTES}")
print(f"  Max notes per melody: {MAX_NOTES}")
print(f"  Transposition augmentation: ±{TRANSPOSE_RANGE} semitones\n")

# Load MIDI files by artist
artist_data = defaultdict(list)

for artist, folder_path in ARTIST_FOLDERS.items():
    if not os.path.exists(folder_path):
        print(f"  ⚠ Skipping {artist}: folder not found ({folder_path})")
        continue
    
    midi_files = glob.glob(os.path.join(folder_path, "*.mid"))
    print(f"  📁 {artist}: found {len(midi_files)} MIDI files")
    
    for midi_file in midi_files:
        try:
            sequence = midi_io.midi_file_to_note_sequence(midi_file)
            melody = extract_melody_from_sequence(sequence)
            
            if melody:
                artist_data[artist].append({
                    'filename': os.path.basename(midi_file),
                    'melody': melody,
                    'length': len(melody),
                    'pitch_range': max(melody) - min(melody)
                })
        except Exception as e:
            pass

print(f"\n✓ Extracted melodies:")
for artist, melodies in artist_data.items():
    print(f"  {artist}: {len(melodies)} melodies")
    if melodies:
        avg_length = np.mean([m['length'] for m in melodies])
        print(f"    Avg length: {avg_length:.0f} notes")


🎼 MIDI Preprocessing Configuration
  Artists to process: ['ABBA']
  Output directory: ./Artist_MIDI/training_data
  Min notes per melody: 16
  Max notes per melody: 1000
  Transposition augmentation: ±12 semitones

  📁 ABBA: found 16 MIDI files

✓ Extracted melodies:
  ABBA: 16 melodies
    Avg length: 375 notes


## Step 2: Create TFRecord Files for Training

In [12]:
def melody_to_note_sequence(melody, tempo=120, velocity=80, start_time=0):
    """Convert a Melody object to a full NoteSequence."""
    sequence = note_seq.NoteSequence()
    
    # Set tempo using tempos field
    sequence.tempos.add(time=0, qpm=tempo)
    
    current_time = start_time
    note_duration = 0.5  # Quarter note
    
    for pitch in melody:
        note = sequence.notes.add()
        note.start_time = current_time
        note.end_time = current_time + note_duration
        note.pitch = int(pitch)
        note.velocity = velocity
        current_time += note_duration
    
    return sequence

def create_tfrecord_for_artist(artist_name, melodies, output_dir, augment=True):
    """
    Create a TFRecord file for an artist's melodies.
    
    Args:
        artist_name: name of the artist
        melodies: list of Melody objects
        output_dir: where to save the TFRecord
        augment: whether to transpose for augmentation
    """
    artist_dir = os.path.join(output_dir, artist_name)
    os.makedirs(artist_dir, exist_ok=True)
    
    tfrecord_path = os.path.join(artist_dir, f'{artist_name}_melodies.tfrecord')
    
    sequences = []
    
    # Add original melodies
    for melody in melodies:
        seq = melody_to_note_sequence(melody)
        sequences.append(seq)
    
    # Add transposed versions for augmentation
    if augment:
        for melody in melodies:
            for transpose_amount in range(-5, 7):
                if transpose_amount == 0:
                    continue
                transposed = transpose_melody(melody, transpose_amount)
                seq = melody_to_note_sequence(transposed)
                sequences.append(seq)
    
    # Write TFRecord
    writer = tf.io.TFRecordWriter(tfrecord_path)
    
    for sequence in sequences:
        sequence_example = sequence.SerializeToString()
        writer.write(sequence_example)
    
    writer.close()
    
    return tfrecord_path, len(sequences)

# Create TFRecord files for each artist
tfrecord_info = {}

print("\n💾 Creating TFRecord files...\n")

for artist, melodies in artist_data.items():
    if not melodies:
        print(f"  ⚠ {artist}: No valid melodies, skipping")
        continue
    
    tfrecord_path, num_sequences = create_tfrecord_for_artist(
        artist,
        [m['melody'] for m in melodies],
        TRAINING_DATA_DIR,
        augment=True
    )
    
    tfrecord_info[artist] = {
        'path': tfrecord_path,
        'original_count': len(melodies),
        'augmented_count': num_sequences
    }
    
    print(f"  ✓ {artist}")
    print(f"    Original melodies: {len(melodies)}")
    print(f"    With augmentation: {num_sequences}")
    print(f"    Saved to: {tfrecord_path}\n")

print("✓ Data preprocessing complete!")
print(f"\nReady for fine-tuning: {list(tfrecord_info.keys())}")


💾 Creating TFRecord files...

  ✓ ABBA
    Original melodies: 16
    With augmentation: 192
    Saved to: ./Artist_MIDI/training_data\ABBA\ABBA_melodies.tfrecord

✓ Data preprocessing complete!

Ready for fine-tuning: ['ABBA']
